Table of contents
=================

<!--ts-->
   * [Pre Reqs](#pre)
   * [Installation](#install)
   * [Setup](#setup)
   * [Getting Started](#start)
   * [Viewing Your Config](#config)
   * [Secure Change](#secure-change)
<!--te-->


Installation      <a class="anchor" id="install"></a>
============
[Git](https://git-scm.com) is a prerequisite for installation. Also, development using either
[`venv`](https://docs.python.org/3/library/venv.html#creating-virtual-environments)
or [Docker](https://docs.docker.com/get-docker/)
is recommended.

`pytos2` CE is currently *not* yet available on [PyPI](https://pypi.org). To install, use
the following [`pip`](https://pypi.org/project/pip/) command:


In [ ]:
# PRODUCTION INSTALL
# %pip install python-dotenv git+https://gitlab.com/tufinps/pytos2-ce

# LOCAL DEVELOPMENT
%pip uninstall pytos2-ce -y
%pip install python-dotenv
%pip install ../../pytos2-ce

Setup      <a class="anchor" id="setup"></a>
=====
`pytos2` CE recommends setting the following environment variables while using this library. They can either be set directly in the environment using a method of your choice, or you can use [python-dotenv](https://saurabh-kumar.com/python-dotenv/).

- `HOST`: The hostname 
- `USER`: The username 
- `PASS`: The password

These values can be editted in the `.env` file or overwritten in-line in the example below

Getting Started      <a class="anchor" id="start"></a>
===============

In [ ]:
# %load init.py
from dotenv import dotenv_values
from pytos2.secureapp import Sa as SecureApp
from pytos2.securechange import Scw as SecureChange
from pytos2.securetrack import St as SecureTrack


# Loads our config
config = dotenv_values(".env")

# Override or Use Config Values
# If changes are made, rerun this cell to see the changes
# Strings must be empty to use config - any value in string will be truthy
HOST = "" or config["AURORA_HOST"]
USER = "" or config["USER"]
PASS = "" or config["PASS"]

# Instantiate Classes
secure_app = SecureApp(HOST, USER, PASS)
secure_change = SecureChange(HOST, USER, PASS)
secure_track = SecureTrack(HOST, USER, PASS)

Configuration      <a class="anchor" id="config"></a>
===================

In [ ]:
print(HOST)
print(USER)
print(PASS)

Secure Change      <a class="anchor" id="secure-change"></a>
=============

### Get Excluded Devices

```python
    .get_excluded_device_ids(
        show_all: Optional[bool] = None
    )
```

In [ ]:
show_all = True
excluded_devices = secure_change.get_excluded_device_ids(show_all)

print(excluded_devices)

### Get Ticket By ID

```python
   .get_ticket(
      _id: int
   )
```

In [ ]:
# Get Ticket by ID # 674
ticket = secure_change.get_ticket(993)

print(ticket)

### Get Tickets

```python
   .get_tickets(
        status: Optional[Union['pytos2.securechange.ticket.TicketStatus',List['pytos2.securechange.ticket.TicketStatus']]] = None,
        start: Optional[int] = None, 
        descending: Optional[bool] = None,
        expand_links: Optional[bool] = None
    )
```

In [ ]:
from pytos2.securechange.ticket import TicketStatus

status = TicketStatus.RESOLVED

tickets = secure_change.get_tickets(status)

print(tickets)

### Get Users

```python

```

In [ ]:
users = secure_change.get_users()

for index, user in enumerate(users):
    print(f"User #{index + 1}")
    print(user)

### Get User

```python
    .get_user(
        identifier: Union[str, int], expand: bool = False
    )
```

#### Get User By ID

> by default, if type is int it will search by id

In [ ]:
# Provide an integer
user_by_id = secure_change.get_user(1)

print(user_by_id)

#### Get User By Name

> by default, if type is string it will search by name

In [ ]:
# Provide a string
user_by_name = secure_change.get_user("r")

print(user_by_name)

### Reassign Ticket

```python
    .reassign_ticket(
        ticket, user, step: Union[None, 'Step', int, str] = None, task: Union[None,
        'Task', int] = None, comment = ''
    )
```

In [ ]:
ticket = secure_change.get_ticket(674)
user = secure_change.get_user("r")
reassigned_ticket = secure_change.reassign_ticket(
    ticket, user, comment="For Jupyter Demo"
)

print(reassigned_ticket)

### Ticket Search

```python
    .ticket_search(
        subject: Optional[str] = None, 
        requester: Optional[str] = None,
        group: Optional[str] = None, 
        assigned_to: Optional[str] = None,
        priority: Optional[str] = None, 
        status: Optional[Union[str,'pytos2.securechange.ticket.Task.Status']] = None,
        sla_status: Optional[Union[str,'pytos2.securechange.ticket.Ticket.SlaStatus']] = None,
        field_name: Optional[str] = None, 
        field_value: Optional[str] = None,
        current_step: Optional[str] = None, 
        expiration_date_from: Optional[str] = None,
        expiration_date_to: Optional[str] = None, 
        domain_name: Optional[str] = None
    )
```

In [ ]:
tickets = secure_change.ticket_search(assigned_to="r")
filtered_list = list(filter(lambda ticket: ticket["id"] == 674, tickets))

print(filtered_list)

### Get Attachment

```python
   .get_attachment(self, file_id: Union[int, str]) -> bytes:
```

In [ ]:
t = secure_change.get_ticket(990)

comment = t.comments[0]
attachment = comment.attachments[0]
attachment_content = secure_change.get_attachment(attachment.uid)

with open(f"Test-{attachment.name}", "wb") as file:
    file.write(attachment_content)

### Add Attachment

```python
   .add_attachment(self, file: str) -> str:
```

In [ ]:
uuid = secure_change.add_attachment("../README.pdf")

print(uuid)

### Add Comment

```python
   .add_comment(
        self,
        ticket_id: Union[int, str],
        step_id: Union[int, str],
        task_id: Union[int, str],
        comment_content: str,
        attachment_uuids: Optional[List[str]],
    ) -> str:
```

In [ ]:
from pytos2.securechange.ticket import Ticket, Comment, Attachment

ticket: Ticket = secure_change.get_ticket(990)
current_step = ticket.current_step
current_task = ticket.current_task

uuid = secure_change.add_attachment("../README.pdf")
comment = "This is a new comment with an attachment"

secure_change.add_comment(ticket.id, current_step.id, current_task.id, comment, [uuid])

### There is also two helpers on the Ticket ###
#   by default the current step and task are used if not provided #

ticket: Ticket = secure_change.get_ticket(993)

ticket.add_comment(comment, [uuid])

c_one = Comment(content="Comment One")
c_two = Comment(content="Comment Two")

ticket.add_comments([c_one, c_two])

### Delete Comment

```python
   delete_comment(self, ticket_id: Union[int, str], comment_id: Union[int, str],) -> None:
```

In [ ]:
ticket: Ticket = secure_change.get_ticket(993)

comment = ticket.comments[-1]  # get last comment

secure_change.delete_comment(ticket.id, comment.id)

### There is also two helpers on the Ticket ###

ticket.delete_comment("230")

ticket.delete_comments([231, 232])

- - -
- - -
- - -


Secure Track      <a class="anchor" id="secure-track"></a>
=============

### Get Devices

```python
   .get_devices(
      cache: Optional[bool] = None, filter: Optional[dict] = None
   )
```

In [ ]:
devices = secure_track.get_devices()

print(devices)

### Get Zones

```python
   .get_zones(
      cache: Optional[bool] = None
   )
```

In [ ]:
zones = secure_track.get_zones()

print(zones)

### Get Zone Subnets

```python
   .get_zone_subnets(
      identifier: Union[int, str, List[int]]
   )
```

In [ ]:
subnets = secure_track.get_zone_subnets(164)

print(subnets)

### Get Zone Descendants

```python
   .get_zone_descendants(
      identifier: Union[int, str, List[int]]
   )
```

In [ ]:
descendants = secure_track.get_zone_descendants(164)

print(descendants)

### Get Zone 

```python
   .get_zone(
      identifier: Union[int, str], cache: Optional[bool] = None
   )
```

In [ ]:
zone = secure_track.get_zone(164)

print(zone)

### Get Domains

```python
   .get_domains(
      cache: Optional[bool] = None
   )
```

In [ ]:
domains = secure_track.get_domains()

print(domains)

### Get Domains

```python
   .get_domain(
      identifier: Union[int, str], cache: Optional[bool] = None
   )
```


In [ ]:
domain = secure_track.get_domain(11, False)

print(domain)

### Add Domain

```python
   .add_domain(
      name: str, description: Optional[str] = None, address: Optional[str] = None
   )
```

In [ ]:
new_domain = secure_track.add_domain("Test")

print(new_domain)

### Update Domain

```python
   .update_domain(
      identifier: Union[int, str], name: Optional[str] = None,
      description: Optional[str] = None, address: Optional[str] = None
   )
```

In [ ]:
# updating something you just created fails to find domain - cache?
updated_domain = secure_track.update_domain(
    9,
    "Updated Toronto Backup",
)

print(updated_domain)

### Get Devices

```python
   .get_device(
      identifier: Union[int, str], cache: Optional[bool] = None
   )
```

In [ ]:
device = secure_track.get_device(38)

print(device)

### Get Network Objects

```python
   .get_network_objects(
      device: Union[int, str], cache: Optional[bool] = None
   )
```

In [ ]:
network_objs = secure_track.get_network_objects(38)

print(network_objs)

### Get Network Object

```python
   .get_network_object(
      name: Optional[str] = None, device: Union[int, str, None] = None,
      uid: Optional[str] = None, cache = True
   )
```

In [ ]:
# this in not intuitive. cant get to work
network_obj = secure_track.get_network_objects("Any", 38)

print(network_obj)

### Get Services

```python
   .get_services(
      device: Union[int, str], cache: Optional[bool] = None
   )
```

In [ ]:
services = secure_track.get_services(38)

print(services)

### Get Service

```python
   .get_service(
      name: Optional[str] = None, device: Union[int, str, None] = None,
      uid: Optional[str] = None, cache = True
   )
```

In [ ]:
service = secure_track.get_service("udp-echo (udp)", 1)

print(service)

### Get NAT Rules

```python
   .get_nat_rules(
      device: Union[str, int, None] = None
   )
```

In [ ]:
# broken too
nat_rules = secure_track.get_nat_rules(38)
# blows up cuz cant kwargify
print(nat_rules)

### Get Rules

```python
   .get_rules(
      device: Union[str, int, None] = None, revision: Union[int, None] = None,
      rule_uid: Optional[str] = None, uid: Optional[str] = None, documentation: bool = True,
      cache: bool = True
   )
```

In [ ]:
rules = secure_track.get_rules(1)

print(rules)

### Get Rule Documentation

```python
   .get_rule_documentation(
      : 'St', device: Union[str, int], rule: Union[int, SecurityRule]
   )
```

In [ ]:
rule_docs = secure_track.get_rule_documentation(1, 2484)

print(rule_docs)

### Get Shadow Rules for Devices

```python
   .get_shadowing_rules_for_device(
      : 'St', device: str, rules: Iterator[str]
   )
```

In [ ]:
rules = secure_track.get_rules(1)
# how to get string from List[SecurityRule]
shadow_rules = secure_track.get_shadowing_rules_for_device(1, rules)

print(shadow_rules)

### Update Rules Documentation

```python
   .update_rule_documentation(
      device: Union[str, int], rule: Union[int, SecurityRule],
      rule_documentation: Documentation
   )
```

In [ ]:
docs = secure_track.get_rule_documentation(1, 2484)

rule_docs = secure_track.update_rule_documentation(1, 2484, docs)

print(rule_docs)

### Get Revision

```python
   .get_revision(
      revision: int, cache: bool = True
   )
```

In [ ]:
rev = secure_track.get_revision(1)

print(rev)

### Get Latest Revision

```python
   .get_latest_revision(
      device: Union[str, int]
   )
```

In [ ]:
latest_rev = secure_track.get_latest_revision(1)

print(latest_rev)

### Get Revisions

```python
   .get_revisions(
      device: Union[str, int], cache: bool = True
   )
```

In [ ]:
revs = secure_track.get_revisions(1)

print(revs)

### Search Rules

```python
   .search_rules(
      : 'St', text: Optional[str] = None, devices: Union[Union[str, int],
      List[Union[str, int]]] = None, context: Optional[int] = None,
      shadowed: Optional[bool] = None, expiration_date: Optional[Union[Emptiness,
      date]] = None, certification_expiration_date: Optional[Union[Emptiness,
      date]] = None, comment: Optional[Emptiness] = None, **search_text_params: Dict[str,
      Union[str, bool, List[Union[str, bool]]]]
   )
```

In [ ]:
rules = secure_track.search_rules()

print(rules)

### Rule Search

```python
   .rule_search(
      *args: tuple, **kwargs: dict
   )
```

In [ ]:
# whats the difference - this looks like it should be private
rules = secure_track.rule_search(1)

print(rules)

### Update Documentation

```python
   .update_documentation(
      device_id: int, rule_id: int, rule_doc: Documentation
   )
```

In [ ]:
# whats the difference between this and update_rule_documation
docs = secure_track.get_rule_documentation(1, 2484)
# keep getting bad request - what should documentation look like?
rule_docs = secure_track.update_documentation(1, 2484, docs)

print(rule_docs)

### Get Device Policies

```python
   .get_device_policies(
      device: Union[int, str]
   )
```

In [ ]:
policies = secure_track.get_device_policies(1)

print(policies)

### Get Device Policy

```python
   .get_device_policy(
      device: Union[int, str], policy: str
   )
```

In [ ]:
# cant find a device with a policy
policy = secure_track.get_device_policy(1, "")

print(policy)

### Get Interfaces

```python
   .get_interfaces(
      device_id: int
   )
```

In [ ]:
# cant find a device with a policy
intfs = secure_track.get_interfaces(38)
# blows up cuz cant kwargify
print(intfs)

### Get Bindable Objects

```python
   .get_bindable_objects(
      device_id: int
   )
```

In [ ]:
objs = secure_track.get_bindable_objects(1)
# blows up cuz cant kwargify
print(objs)

### Get Topology Interfaces

```python
   .get_topology_interfaces(
      device_id: int, is_generic: Optional[int] = 0
   )
```

In [ ]:
intfs = secure_track.get_topology_interfaces(1)

print(intfs)

### Get Generic Devices

```python
   .get_generic_devices(
      name: Optional[str] = None, context: Optional[int] = None, cache: bool = True
   )
```

In [ ]:
devices = secure_track.get_generic_devices()
# this doesnt update after delete - cache?
print(devices)

### Add Generic Device

```python
   .add_generic_device(
      name: str, configuration: Union[BytesIO, str], update_topology: bool = False,
      customer_id: Optional[int] = None
   )
```

In [ ]:
device = secure_track.add_generic_device("Test", "")
# keep getting bad request - what should config look like - its not a get generic device
print(device)

### Remove Generic Devices

```python
   .delete_generic_device(
      identifier: Union[int, str], update_topology: bool = False
   )
```

In [ ]:
device = secure_track.delete_generic_device(6)

print(device)

### Import Generic Device

```python
   .import_generic_device(
      name: str, configuration: Union[BytesIO, str], update_topology: bool = False,
      customer_id: Optional[int] = None
   )
```

In [ ]:
# what the difference between this and add?
device = secure_track.import_generic_device("Test", "")
# keep getting bad request - what should config look like - its not a get generic device
print(device)

### Sync Topology

```python
   .sync_topology(
      full_sync: bool = False
   )
```

In [ ]:
sync = secure_track.sync_topology()

print(sync)

### Get Topology Sync Status

```python
    .get_topology_sync_status()
```

In [ ]:
status = secure_track.get_topology_sync_status()

print(status)

### Add Generic Interface

```python
    .add_generic_interface(
        interface: dict
    ) -> None:
```

In [ ]:
interface = {
    "mgmtId": 1,
    "name": "eth1.5",
    "ip": "10.100.110.99",
    "mask": "255.255.255.0",
    "mpls": False,
    "unnumbered": False,
}
secure_track.add_generic_interface(interface)

### Add Mulitple Generic Interfaces

```python
    .add_generic_interfaces(
        interfaces: List[dict]
    ) -> None:
```

In [ ]:
interfaces = [
    {
        "mgmtId": 1,
        "name": "newApo1",
        "ip": "100.103.33.31",
        "mask": "255.255.255.0",
        "vrf": "V101-PAL",
        "mpls": False,
        "unnumbered": False,
        "type": "external",
    },
    {
        "mgmtId": 1,
        "name": "newApo2",
        "ip": "100.103.33.32",
        "mask": "255.255.255.0",
        "vrf": "V101-PA",
        "mpls": False,
        "unnumbered": False,
        "type": "external",
    },
    {
        "mgmtId": 1,
        "name": "newApo3",
        "ip": "100.103.33.33",
        "mask": "255.255.255.0",
        "vrf": "V101-PA",
        "mpls": False,
        "unnumbered": False,
        "type": "internal",
    },
]
secure_track.add_generic_interfaces(interfaces)

### Get Generic Interface by ID

```python
    .get_generic_interface(
        device_id: Union[int, str]
    ) -> GenericTopologyInterface:
```

In [ ]:
secure_track.get_generic_interface(42)
secure_track.get_generic_interface("42")

### Get Generic Interfaces by Management ID

```python
    .get_generic_interfaces(
        mgmt_id: Union[int, str]
    ) -> GenericTopologyInterface:
```

In [ ]:
secure_track.get_generic_interfaces(1)
secure_track.get_generic_interfaces("1")

### Update Generic Interface

```python
    .update_generic_interface(
        interface: dict
    ) -> None:
```

In [ ]:
interface = {
    "id": 41,
    "mgmtId": 1,
    "name": "eth1.5",
    "ip": "10.100.110.99",
    "mask": "255.255.255.0",
    "mpls": False,
    "unnumbered": False,
}
secure_track.update_generic_interface(interface)

### Update Multiple Generic Interfaces

```python
    .update_generic_interfaces(
        interfaces: List[dict]
    ) -> None:
```

In [ ]:
interfaces = [
    {
        "id": 42,
        "mgmtId": 1,
        "name": "newApo1",
        "ip": "100.103.33.31",
        "mask": "255.255.255.0",
        "vrf": "V101-PAL",
        "mpls": False,
        "unnumbered": False,
        "type": "external",
    },
    {
        "id": 43,
        "mgmtId": 1,
        "name": "newApo2",
        "ip": "100.103.33.32",
        "mask": "255.255.255.0",
        "vrf": "V101-PA",
        "mpls": False,
        "unnumbered": False,
        "type": "external",
    },
    {
        "id": 44,
        "mgmtId": 1,
        "name": "newApo3",
        "ip": "100.103.33.33",
        "mask": "255.255.255.0",
        "vrf": "V101-PA",
        "mpls": False,
        "unnumbered": False,
        "type": "internal",
    },
]
secure_track.update_generic_interfaces(interfaces)

### Delete Generic Inteface by ID

```python
    .delete_generic_interface(
        device_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_interface(41)

### Delete Generic Intefaces by Management ID

```python
    .delete_generic_interfaces(
        mgmt_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_interfaces(1)

### Add Generic Route

```python
    .add_generic_route(
        route: dict
    ) -> None:
```

In [ ]:
route = {
    "mgmtId": 92,
    "destination": "10.4.4.4",
    "mask": "255.0.0.0",
    "interfaceName": "",
    "nextHop": "AA",
    "nextHopType": "VR",
    "vrf": "V102-YO",
}
secure_track.add_generic_route(route)

### Add Generic Routes

```python
    .add_generic_routes(
        routes: List[dict]
    ) -> None:
```

In [ ]:
routes = [
    {
        "mgmtId": 92,
        "destination": "10.4.4.4",
        "mask": "255.0.0.0",
        "interfaceName": "",
        "nextHop": "AA",
        "nextHopType": "VR",
        "vrf": "V102-YO",
    },
    {
        "mgmtId": 92,
        "destination": "10.4.5.5",
        "mask": "255.0.0.0",
        "interfaceName": "eeee",
        "nextHop": "",
        "nextHopType": "IP",
        "vrf": "V101-PA",
    },
]
secure_track.add_generic_routes(routes)

### Get Generic Route

```python
    .get_generic_route(
        int_id: Union[int, str]
    ) -> GenericRoute:
```

In [ ]:
secure_track.get_generic_route(3)

### Get Generic Routes

```python
    .get_generic_routes(
        mgmt_id: Union[int, str]
    ) -> List[GenericRoute]:
```

In [ ]:
secure_track.get_generic_routes(92)

### Update Generic Route

```python
    .update_generic_route(
        route: dict
    ) -> None:
```

In [ ]:
route = {
    "mgmtId": 92,
    "destination": "10.4.4.4",
    "mask": "255.0.0.0",
    "interfaceName": "",
    "nextHop": "AA",
    "nextHopType": "VR",
    "vrf": "V102-YO",
}
secure_track.update_generic_route(route)

### Update Generic Routes

```python
    .update_generic_routes(
        routes: List[dict]
    ) -> None:
```

In [ ]:
routes = [
    {
        "mgmtId": 92,
        "destination": "10.4.4.4",
        "mask": "255.0.0.0",
        "interfaceName": "",
        "nextHop": "AA",
        "nextHopType": "VR",
        "vrf": "V102-YO",
    },
    {
        "mgmtId": 92,
        "destination": "10.4.5.5",
        "mask": "255.0.0.0",
        "interfaceName": "eeee",
        "nextHop": "",
        "nextHopType": "IP",
        "vrf": "V101-PA",
    },
]
secure_track.update_generic_routes(routes)

### Delete Generic Route

```python
    .delete_generic_route(
        int_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_route(3)

### Delete Generic Routes

```python
    .delete_generic_routes(
        mgmt_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_routes(92)

### Add Generic VPN

```python
    .add_generic_vpn(
        vpn: dict
    ) -> None:
```

In [ ]:
vpn = {
    "generic": True,
    "deviceId": 182,
    "interfaceName": "777",
    "tunnelSourceIpAddr": "1.1.1.177",
    "tunnelDestIpAddr": "1.1.1.188",
}
secure_track.add_generic_vpn(vpn)

### Add Generic VPNs

```python
    .add_generic_vpns(
        vpns: List[dict]
    ) -> None:
```

In [ ]:
vpns = [
    {
        "generic": True,
        "deviceId": 182,
        "interfaceName": "new33",
        "vpnName": None,
        "tunnelSourceIpAddr": "3.3.3.33",
        "tunnelDestIpAddr": "1.1.1.11",
    },
    {
        "generic": False,
        "deviceId": 182,
        "interfaceName": "new111",
        "vpnName": None,
        "tunnelSourceIpAddr": "1.1.1.11",
        "tunnelDestIpAddr": "3.3.3.33",
    },
]
secure_track.add_generic_vpns(vpns)

### Get Generic VPN

```python
    .get_generic_vpn(
        int_id: Union[int, str]
    ) -> GenericVpn:
```

In [ ]:
secure_track.get_generic_vpn(1)

### Get Generic VPNs

```python
    .get_generic_vpns(
        device_id: Union[int, str]
    ) -> List[GenericVpn]:
```

In [ ]:
secure_track.get_generic_vpns(182)

### Update Generic VPN

```python
    .update_generic_vpn(
        vpn: dict
    ) -> None:
```

In [ ]:
vpn = {
    "generic": True,
    "deviceId": 182,
    "interfaceName": "777",
    "tunnelSourceIpAddr": "1.1.1.177",
    "tunnelDestIpAddr": "1.1.1.188",
}
secure_track.update_generic_vpn(vpn)

### Update Generic VPNs

```python
    .update_generic_vpns(
        vpns: List[dict]
    ) -> None:
```

In [ ]:
vpns = [
    {
        "generic": True,
        "deviceId": 182,
        "interfaceName": "new33",
        "vpnName": None,
        "tunnelSourceIpAddr": "3.3.3.33",
        "tunnelDestIpAddr": "1.1.1.11",
    },
    {
        "generic": False,
        "deviceId": 182,
        "interfaceName": "new111",
        "vpnName": None,
        "tunnelSourceIpAddr": "1.1.1.11",
        "tunnelDestIpAddr": "3.3.3.33",
    },
]
secure_track.update_generic_vpns(vpns)

### Delete Generic VPN

```python
    .delete_generic_vpn(
        int_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_vpn(3)

### Delete Generic VPNs

```python
    .delete_generic_vpns(
        mgmt_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_vpns(92)

### Add Generic Transparent Firewalls

```python
    .add_generic_transparent_firewalls(
        firewalls: List[dict]
    ) -> None:
```

In [ ]:
firewalls = [
    {
        "outputL3DeviceId": 22,
        "outputL3IsGenericDevice": False,
        "outputL3InterfaceName": "FastEthernet0/0",
        "layer2DeviceId": 9,
        "inputL2InterfaceName": "inside",
        "outputL2InterfaceName": "outside",
        "inputL3DeviceId": 1,
        "inputL3IsGenericDevice": True,
        "inputL3InterfaceName": "interface2",
    },
    {
        "outputL3DeviceId": 23,
        "outputL3IsGenericDevice": False,
        "outputL3InterfaceName": "FastEthernet2/0",
        "layer2DeviceId": 9,
        "inputL2InterfaceName": "inside",
        "outputL2InterfaceName": "outside",
        "inputL3DeviceId": 17,
        "inputL3IsGenericDevice": False,
        "inputL3InterfaceName": "mgmt",
    },
]
secure_track.add_generic_transparent_firewalls(firewalls)

### Get Generic Transparent Firewalls

```python
    .get_generic_transparent_firewalls(
        device_id: Union[int, str]
    ) -> List[GenericTransparentFirewall]:
```

In [ ]:
secure_track.get_generic_transparent_firewalls(92)

### Update Generic Transparent Firewalls

```python
    .update_generic_transparent_firewalls(
        firewalls: List[dict]
    ) -> None:
```

In [ ]:
firewalls = [
    {
        "outputL3DeviceId": 22,
        "outputL3IsGenericDevice": False,
        "outputL3InterfaceName": "FastEthernet0/0",
        "layer2DeviceId": 9,
        "inputL2InterfaceName": "inside",
        "outputL2InterfaceName": "outside",
        "inputL3DeviceId": 1,
        "inputL3IsGenericDevice": True,
        "inputL3InterfaceName": "interface2",
    },
    {
        "outputL3DeviceId": 23,
        "outputL3IsGenericDevice": False,
        "outputL3InterfaceName": "FastEthernet2/0",
        "layer2DeviceId": 9,
        "inputL2InterfaceName": "inside",
        "outputL2InterfaceName": "outside",
        "inputL3DeviceId": 17,
        "inputL3IsGenericDevice": False,
        "inputL3InterfaceName": "mgmt",
    },
]
secure_track.update_generic_transparent_firewalls(firewalls)

### Delete Generic Transparent Firewalls

```python
    .delete_generic_transparent_firewall(
        layer_2_data_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_transparent_firewall(3)

### Delete Generic Transparent Firewalls

```python
    .delete_generic_transparent_firewalls(
        device_id: Union[int, str]
    ) -> None:
```

In [ ]:
secure_track.delete_generic_transparent_firewalls(92)